# 練習

- https://www.google.com/recaptcha/demo/recaptcha
- 透過 google reCAPTCHA demo 生成圖片
- 將 reCAPTCHA 的圖片抓下來

In [16]:
import os
import hashlib
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from fake_useragent import UserAgent
from PIL import Image

url = 'https://www.google.com/recaptcha/demo/recaptcha'
fu = UserAgent()

In [28]:
class ReloadImage(object):
    def __init__(self, locator, xpath, compare):
        self.locator = locator
        self.xpath = xpath
        self.compare = compare
    def __call__(self, driver):
        element = driver.find_element(self.locator, self.xpath)
        attr = element.get_attribute('src')
        if attr != self.compare:
            return element
        else:
            return False

In [39]:
results = os.path.abspath('../results')
if not os.path.exists(results):
    os.makedirs(results)

try:
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()
    wait = WebDriverWait(driver, 10)
    compare_url = ''
    
    for i in range(5):
        # get image
        #img_el = driver.find_element(By.XPATH, '//div[@id="recaptcha_image"]/img')
        img_el = wait.until(ReloadImage(By.XPATH, '//div[@id="recaptcha_image"]/img', compare_url))
        img_url = img_el.get_attribute('src')
        img_filename = hashlib.md5(img_url.encode('utf-8')).hexdigest()
        compare_url = img_url

        headers = {'User-Agent': fu.random}
        img_resp = requests.get(img_url, stream=True, headers=headers)
        img = Image.open(img_resp.raw)
        img_filename = '{}.{}'.format(img_filename, img.format)
        img_filename = os.path.join(results, img_filename)
        img.save(img_filename)
        print('Save img - {}'.format(img_filename))
        
        # re-generate image
        btn_refresh = driver.find_element(By.XPATH, '//*[@id="recaptcha_reload_btn"]').click()
        print('click')
        

except Exception as e:
    print(e)
finally:
    driver.quit()

Save img - /home/dirl/github/Python-Crawling-Tutorial/results/dcc521eea7e48f005685d58e770fc9be.JPEG
click
Save img - /home/dirl/github/Python-Crawling-Tutorial/results/6cf5653979671789ef664c9122f83f6f.JPEG
click
Message: chrome not reachable
  (Session info: chrome=64.0.3282.119)
  (Driver info: chromedriver=2.35.528139 (47ead77cb35ad2a9a83248b292151462a66cd881),platform=Linux 4.13.0-26-generic x86_64)

